In [1]:
#!pip uninstall tweepy -y
#!pip install geotext

In [2]:
#!pip3 install tweepy

In [3]:
import tweepy
import webbrowser
import time
import os

from dotenv import dotenv_values
from dotenv import load_dotenv

import pickle

import pandas as pd
import numpy as np
import re


In [4]:
load_dotenv()

consumer_key = os.getenv("CONSUMER_KEY")
consumer_secret = os.getenv("CONSUMER_SECRET")
access_token = os.getenv("ACCESS_TOKEN")
access_token_secret = os.getenv("ACCESS_TOKEN_SECRET")
bearer_token = os.getenv('BEARER_TOKEN')

In [5]:
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

In [6]:
query =  (' "robert malone" -is:retweet lang:en OR \
            "dr malone" -is:retweet lang:en')

max_results = 100
place_fields = ['full_name',"id","country","country_code","geo","name","place_type"]
tweet_fields = ["created_at","text","source","public_metrics","lang","geo","context_annotations"]
user_fields = ["name", "username",'public_metrics',"location","verified","description"]
expansions= 'author_id'


In [6]:
# check number of tweets in 7 days
query =  (' "dr malone " -is:retweet lang:en OR \
            "robert malone " -is:retweet lang:en')
count_tweet = client.get_recent_tweets_count(query)
tweet_numbers = pd.DataFrame(count_tweet.data)
tweet_numbers.tweet_count.sum()

25756

In [8]:
# The code transform tweet to dataframe use the following link as reference: https://github.com/jdfoote/Intro-to-Programming-and-Data-Science/blob/fall2021/extra_topics/twitter_v2_example.ipynb


if os.path.exists("tweets_malone.pkl") == False:
    
    tweets_malone = []
    file_name = 'tweets_malone.txt'

    with open(file_name, 'a+') as filehandle:

        for response in tweepy.Paginator(client.search_recent_tweets,query = query,
                                    expansions = expansions,
                                    place_fields = place_fields, 
                                    tweet_fields = tweet_fields,
                                    user_fields = user_fields,
                                    max_results = max_results,limit=500):
            time.sleep(1)
            tweets_malone.append(response)
            for tweet in response :
                filehandle.write('%s\n' % tweet)
    
    result = []
    user_dict = {}
    # Loop through each response object
    for response in tweets_malone:
        # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
        for user in response.includes['users']:
            user_dict[user.id] = {'username': user.username, 
                                  'followers': user.public_metrics['followers_count'],
                                  'followings':user.public_metrics['following_count'],
                                  'tweets': user.public_metrics['tweet_count'],
                                  'description': user.description,
                                  'location': user.location
                                 }
        for tweet in response.data:
            # For each tweet, find the author's information
            author_info = user_dict[tweet.author_id]
            # Put all of the information we want to keep in a single dictionary for each tweet
            result.append({'author_id': tweet.author_id, 
                           'username': author_info['username'],
                           'author_followers': author_info['followers'],
                           'author_followings': author_info['followings'],
                           'author_tweets': author_info['tweets'],
                           'author_description': author_info['description'],
                           'author_location': author_info['location'],
                           'text': tweet.text,
                           'created_at': tweet.created_at,
                           'retweets': tweet.public_metrics['retweet_count'],
                           'replies': tweet.public_metrics['reply_count'],
                           'likes': tweet.public_metrics['like_count'],
                           'quote_count': tweet.public_metrics['quote_count']
                          })
    
    df_tweets_malone = pd.DataFrame(result)
    
    with open('tweets_malone.pkl','wb') as file:
        pickle.dump(df_tweets_malone,file)
                
else:
    df_tweet_copy= pd.read_pickle('tweets_malone.pkl')


In [12]:
df_tweet = df_tweet_copy.copy()

In [13]:
df_tweet.shape,df_tweet.columns
df_tweet.head()

,author_id,username,author_followers,author_followings,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count
0,1316931193,thekapnobatai,12,88,5820,Connecting Through The Clouds,None,"Dr. Robert Malone: ""The government is now lawl...",2022-01-14 17:47:15+00:00,0,0,0,0
1,136772697,wgaryjohnson,142,893,6566,,None,@jessicamalaty @Spotify @joerogan I'm sure Rog...,2022-01-14 17:47:02+00:00,0,0,0,0
2,867042174,chemTrailActivi,2106,2790,28117,Meeting like-minded citizens who'll fight for ...,"Arizona, USA",“How #Melatonin Prevents #Sepsis”\nMelatonin S...,2022-01-14 17:46:52+00:00,0,0,0,0
3,1377753958933000192,DFoe99,104,703,8636,"If you're a liberal, LOL = You're a fucking id...",None,@jessicamalaty @bethlinas @DrKatEpi I would re...,2022-01-14 17:46:39+00:00,0,0,0,0
4,17266544,bourbonroad,3297,4999,131767,"Screenwriter, Producer, Marketing Manager, Singer","Montclair, CA",@bart_e_anderson @rebel_ess @mtaibbi Hmmm...ha...,2022-01-14 17:45:58+00:00,0,0,0,0


In [14]:
df_tweet.drop_duplicates(subset=['text'])
df_tweet.shape

(22938, 13)